In [2]:
from dotenv import load_dotenv
load_dotenv("./.env", override=True)

True

In [3]:
from pydantic_settings import BaseSettings

class ModelConfig(BaseSettings):
    gemini_api_key: str
    model_name: str
    temperature: float

In [4]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

In [5]:
class ChatModel(ChatGoogleGenerativeAI):
    def __init__(self, config: ModelConfig):
        super().__init__(
            model=config.model_name,
            temperature=config.temperature,
            api_key=config.gemini_api_key
        )

In [6]:
from pydantic import BaseModel
from typing import List, Dict, Any
import re, uuid

In [7]:
class ActionState(BaseModel):
    transcript: str
    chunks: List[Dict[str, Any]] = []
    action_items: List[Dict[str, Any]] = []

In [8]:
def split_into_chunks(text: str, max_chars: int = 3000) -> List[Dict[str, Any]]:
    """
    Splits transcripts by speaker lines, keeps names + text
    Return list of dicts {id, speaker, text}"""

    chunks = []

    while True:
        if text:
            txt_chunk = text[:max_chars]
            text = text[max_chars:]
        else:
            print("No more text to process")
            break

        block = re.split(r'(?=\n[A-Z][a-z]+:)', txt_chunk)
        if len(block) > 1:
            chunk = " ".join(block[:-1])
            text = block[-1] + text
        else:
            chunk  = "".join(block)
            text = None

        if chunk:
            chunks.append(chunk)

        if not text:
            break

    return chunks

In [9]:
def ingest_transcript(state: ActionState) -> ActionState:
    chunks = split_into_chunks(state.transcript)
    state.chunks = [{"id": str(uuid.uuid4()), "text": chunk} for chunk in chunks]
    return state

In [10]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [11]:
def extract_actions(state: ActionState) -> ActionState:
    
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(
                """
                You are an Meeting Minutes Assistant.
                Given a meeting transcript, you will extract actionable tasks from the transcript.
                Important: The 'owner' is the person RESPONSIBLE for the task, not the person who mentioned it.
                """
            ),
            HumanMessagePromptTemplate.from_template(
                """
                Given the following meeting transcript, extract all actionable tasks.
                
                Transcript:
                {transcript}
                
                Return a JSON array in the following format:
                [
                  {{"owner": "person_name", "task": "task_description", "due_date": "date_or_null"}},
                  {{"owner": "person_name", "task": "task_description", "due_date": "date_or_null"}}
                ]
                
                Only return the JSON array, no other text or markdown formatting.
                """
            )
        ]
    )

    chat = ChatModel(ModelConfig())
    lcel = (
        prompt
        | chat
    )


    action_items = []
    for chunk in state.chunks:
        text = chunk["text"]
        response = lcel.invoke({"transcript": text})

        try:
            import json
            # Parse the JSON response and extend the action_items list
            parsed_items = json.loads(response.content)
            if isinstance(parsed_items, list):
                action_items.extend(parsed_items)
            else:
                action_items.append(parsed_items)
        except json.JSONDecodeError:
            action_items.append({"error": "Failed to parse JSON response", "response": response.content[:200]})

    state.action_items = action_items

    return state
    

In [12]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(
    state_schema=ActionState,
)

In [13]:
workflow.add_node("ingest_transcript", ingest_transcript)
workflow.add_node("extract_actions", extract_actions)

workflow.add_edge(START, "ingest_transcript")
workflow.add_edge("ingest_transcript", "extract_actions")
workflow.add_edge("extract_actions", END)

meeting_agent = workflow.compile()

In [14]:
transcript = """
    Alice: We need the quarterly report ready by Friday.
    Bob: I'll prepare the financial section.
    Priya: I'll set up a follow-up meeting next week.
    """

output = meeting_agent.invoke({"transcript": transcript})

In [15]:
# Display the results
print("=== Meeting Minutes Processing Results ===")
print(f"\nTranscript processed into {len(output['chunks'])} chunks")
print(f"Extracted {len(output['action_items'])} action items:")
print("\n--- Action Items ---")
for i, item in enumerate(output['action_items'], 1):
    if 'error' in item:
        print(f"{i}. ERROR: {item}")
    else:
        print(f"{i}. Owner: {item['owner']}")
        print(f"   Task: {item['task']}")
        print(f"   Due Date: {item['due_date']}")
        print()

=== Meeting Minutes Processing Results ===

Transcript processed into 1 chunks
Extracted 2 action items:

--- Action Items ---
1. Owner: Bob
   Task: Prepare the financial section (of the quarterly report)
   Due Date: Friday

2. Owner: Priya
   Task: Set up a follow-up meeting
   Due Date: None

